# Model description

**Pose landmarker model**

The MediaPipe Pose Landmarker task lets you detect landmarks of human bodies in an image or video. You can use this task to identify key body locations, analyze posture, and categorize movements. This task uses machine learning (ML) models that work with single images or video. The task outputs body pose landmarks in image coordinates and in 3-dimensional world coordinates.

The pose landmarker model tracks 33 body landmark locations, representing the approximate location of the following body parts:

0 - nose
1 - left eye (inner)
2 - left eye
3 - left eye (outer)
4 - right eye (inner)
5 - right eye
6 - right eye (outer)
7 - left ear
8 - right ear
9 - mouth (left)
10 - mouth (right)
11 - left shoulder
12 - right shoulder
13 - left elbow
14 - right elbow
15 - left wrist
16 - right wrist
17 - left pinky
18 - right pinky
19 - left index
20 - right index
21 - left thumb
22 - right thumb
23 - left hip
24 - right hip
25 - left knee
26 - right knee
27 - left ankle
28 - right ankle
29 - left heel
30 - right heel
31 - left foot index
32 - right foot index

<img src="https://developers.google.com/static/mediapipe/images/solutions/pose_landmarks_index.png" alt="Image" width="400">

By calculating the angle between three key points (shoulder - elbow - wrist) a curl excercise reps counter can be implemented.

# Initialization

In [ ]:
# !pip install opencv-python mediapipe

In [ ]:
# Load libraries
from abc import ABC, abstractmethod
import cv2
import mediapipe as mp
import numpy as np
import contextlib
from typing import List, Tuple, Union, Optional

In [ ]:
# Load MediaPipe pose model
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Create a Counter base class
class Counter(ABC):
    """
    A base class for Counter objects.

    Attributes
    ----------
    counter : int
        The current count.
    state : str
        The current state of the counter.
    landmarks : list
        The landmarks of the body.
    """
    def __init__(self, state = None, output = None):
        self.counter = 0
        self.landmarks = None
        self.state = state
        self.output = output

    @abstractmethod
    def make_calculations(self):
        """
        Takes care of the calculations needed to count.
        """
        ...

    @abstractmethod
    def count(self):
        """
        Takes care of counting logic like incrementing, decrementing, and resetting the counter.
        """
        ...

    def run(self, landmarks: List[mp_pose.PoseLandmark]) -> None:
        """
        Args:
            landmarks (List[mp_pose.PoseLandmark]): List of landmarks from the pose model
        """
        self.landmarks = landmarks
        self.make_calculations()
        self.count()

    def increment(self):
        self.counter += 1

    def decrement(self):
        self.counter -= 1

    def reset(self):
        self.counter = 0

    def __repr__(self):
        if self.output:
            return str(self.output)
        else:
            return str(f"Counter: {str(self.counter)} State: {str(self.state)}")

In [ ]:
# Create counters utils class
class CounterUtils:
    """
    Class with utility functions for counting reps and sets in a workout.
    """
    @staticmethod
    def calculate_angle_3p(a: Tuple[float, float], b: Tuple[float, float], c: Tuple[float, float]) -> int:
        """
        Calculates the angle between three given points.

        Args:
            a (Tuple[float, float]): First point
            b (Tuple[float, float]): Second point
            c (Tuple[float, float]): Third point

        Returns:
            int: Angle between the three given points
        """
        # Calculating the vectors ab and bc
        ab = np.array([a[0] - b[0], a[1] - b[1]])
        bc = np.array([c[0] - b[0], c[1] - b[1]])

        # Calculating the cosine of the angle using the dot product
        cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
        
        # Ensuring the cosine value is within the valid range [-1, 1]
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

        # Calculating the angle in radians and converting it to degrees
        angle = np.arccos(cosine_angle)
        return np.degrees(angle)
    
    def is_visible(self, landmark: mp_pose.PoseLandmark) -> bool:
        return self.landmarks[landmark.value].visibility > 0.5

In [ ]:
# Create curl counter class
class CurlCounter(Counter, CounterUtils):
    def __init__(self, min_angle=30, max_angle=150):
        """
        Class for counting bicep curls.
        
        Args:
            min_angle (int): Minimum angle of the elbow
            max_angle (int): Maximum angle of the elbow
            resting_angle (int): Resting angle of the elbow

        Attributes:
            min_angle (int): Minimum angle of the elbow
            max_angle (int): Maximum angle of the elbow
            resting_angle (int): Resting angle of the elbow
            current_angle (int): Current angle of the elbow
            LEFT_SHOULDER (list): Coordinates of the left shoulder
            LEFT_ELBOW (list): Coordinates of the left elbow
            LEFT_WRIST (list): Coordinates of the left wrist
        """
        super().__init__(state='down', output="Get in position to start reps")
        self.min_angle = min_angle
        self.max_angle = max_angle
        self.resting_angle = max_angle
        self.current_angle = max_angle
        self.LEFT_SHOULDER = None
        self.LEFT_ELBOW = None
        self.LEFT_WRIST = None

    def make_calculations(self):
        """
        Calculates the angle of the elbow and updates the current angle.
        """
        if self.is_visible(mp_pose.PoseLandmark.LEFT_SHOULDER) \
        and self.is_visible(mp_pose.PoseLandmark.LEFT_ELBOW) \
        and self.is_visible(mp_pose.PoseLandmark.LEFT_WRIST):
            self.LEFT_SHOULDER = [self.landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            self.LEFT_ELBOW = [self.landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            self.LEFT_WRIST = [self.landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, self.landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            self.current_angle = self.calculate_angle_3p(self.LEFT_SHOULDER, self.LEFT_ELBOW, self.LEFT_WRIST)
        else:
            self.current_angle = self.resting_angle

    def count(self):
        """
        Counts the number of bicep curls.
        """
        if self.current_angle > self.max_angle and self.state == 'up':
            self.increment()
            self.state = 'down'
        if self.current_angle < self.min_angle and self.state == 'down':
            self.state = 'up'
            self.output = f"Completed reps: {self.counter} Slowly return to starting position"
        if self.state == 'down':
            percentage = round(round(1 - (self.current_angle - self.min_angle) / (self.max_angle - self.min_angle), 2) * 100)
            self.output = f"Completed reps: {self.counter} -> {max(percentage, 0)}% to complete next rep"


In [ ]:
# Create a video capture class, generalizing the code above
class PoseDetectorVideoCapture:
    """
    Class for capturing video from a camera, detecting poses, and displaying the results.

    Attributes
    ----------
    cap : cv2.VideoCapture
        The video capture object.
    screen_width : int
        The width of the screen.
    screen_height : int
        The height of the screen.
    flip : bool
        Whether to flip the video feed horizontally.
    """
    def __init__(self, device: int = 0, flip: bool = False, show_landmarks: bool = True) -> None:
        """
        Args:
            device (int): The device index of the camera to use.
        """
        self.cap = cv2.VideoCapture(device)
        self.screen_width = int(self.cap.get(3))
        self.screen_height = int(self.cap.get(4))
        self.flip = flip
        self.show_landmarks = show_landmarks

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.cap.release()
        cv2.destroyAllWindows()
        
    def run(self, pose_counter: Counter) -> None:
        """
        Runs the pose detector video capture.

        Args:
            pose_counter (Counter): The pose counter to use.
        """
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while self.cap.isOpened():
                _, image = self.cap.read()

                # Flip image horizontally
                if self.flip:
                    image = cv2.flip(image, 1)

                # Recolor feed
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                # Make detection
                results = pose.process(image)

                # Recolor image back to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Extract landmarks
                with contextlib.suppress(Exception):
                    landmarks = results.pose_landmarks.landmark
                    
                    # Run pose counter
                    pose_counter.run(landmarks)

                    # Render pose counter
                    cv2.putText(image, repr(pose_counter), 
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                        (255, 255, 255), 2, cv2.LINE_AA)

                # Render detections
                if self.show_landmarks:
                    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
                    )

                # Show detections
                cv2.imshow('Detection Feed', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

In [ ]:
# Create instance of PoseDetectorVideoCapture
with PoseDetectorVideoCapture(flip=True) as video_capture:
    # Create instance of CurlCounter
    curl_counter = CurlCounter()

    # Run video capture
    video_capture.run(curl_counter)